<a href="https://colab.research.google.com/github/chitzinwin/Generation_SPARQL/blob/discovery/Overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone -b discovery --single-branch https://github.com/chitzinwin/Generation_SPARQL.git

In [ ]:
!nvidia-smi
!nvidia-smi --query-gpu=name --format=csv,noheader

In [27]:
%%capture
%pip install accelerate peft transformers datasets bitsandbytes sentencepiece 

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, AutoPeftModelForCausalLM
import torch

In [36]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    )

In [37]:

model = AutoPeftModelForCausalLM.from_pretrained(
    "./assets/models/wikidata/checkpoint-13500",
    device_map="cuda:0",
    cache_dir="/mnt/d/huggyface",
    offload_folder="/mnt/d/huggyface/offload",
    low_cpu_mem_usage=True,
    quantization_config=bnb_config
)

Loading checkpoint shards:  50%|█████     | 1/2 [07:07<07:07, 427.22s/it]

In [ ]:
from transformers import LlamaTokenizer
tokenizer = LlamaTokenizer.from_pretrained("./assets/models/wikidata/checkpoint-13500")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))


In [ ]:
from text2sparql.format import truncate_batch, extract_first_sparql, load_data_from_file, generate_prompt, prompt_question, prompt_question_contrastivev1, prompt_append_BERN2EL,prompt_append_SpacyWikidataEL,extract_first_sparql_contrastive,extract_first_sparql_bgee
prompt = prompt_question("How many professions were created in 1789?")
inputs = tokenizer(f'<s>[INST] {prompt} [/INST]',return_tensors="pt",  padding=True, truncation=True)

with torch.no_grad():
      outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=512, do_sample=False)
      print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])